In [1]:
import pandas as pd
import numpy as np

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/F1-Fantasy-Predictor/'
elif platform == "darwin":
    path = '~/Documents/GitHub/F1-Fantasy-Predictor/'

In [2]:
merged = pd.read_csv(path+'data/merged.csv')

In [3]:
df = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality'], axis=1)

In [4]:
df.rename(columns={'driver_points': 'driver_points_after', 
    'constructor_points': 'constructor_points_after', 
    'constructor_wins': 'constructor_wins_after', 
    'driver_wins': 'driver_wins_after', 
    'constructor_standings_pos': 'constructor_standings_pos_after',
    'driver_standings_pos': 'driver_standings_pos_after'}, inplace=True)

In [5]:
points_before = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_wins_before': [],
    'driver_points_before': [],
    'driver_standings_before': [],
    'constructor_wins_before': [],
    'constructor_points_before': [],
    'constructor_standings_before': []
}

In [6]:
grouped = df.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            points_before['season'].append(season)
            points_before['round'].append(q.iloc[index]['round'])
            points_before['driver'].append(driver)
            n = q.iloc[index-1]
            if index != 0:
                points_before['driver_points_before'].append(n.driver_points_after)
                points_before['driver_wins_before'].append(n.driver_wins_after)
                points_before['driver_standings_before'].append(n.driver_standings_pos_after)
                points_before['constructor_points_before'].append(n.constructor_points_after)
                points_before['constructor_standings_before'].append(n.constructor_standings_pos_after)
                points_before['constructor_wins_before'].append(n.constructor_wins_after)
            else:
                points_before['driver_points_before'].append(0.0)
                points_before['driver_wins_before'].append(0)
                points_before['driver_standings_before'].append(1)
                points_before['constructor_points_before'].append(0.0)
                points_before['constructor_standings_before'].append(1)
                points_before['constructor_wins_before'].append(0)


points_before = pd.DataFrame(points_before)
points_before.head()

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11460/2145203998.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


,season,round,driver,driver_wins_before,driver_points_before,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before
0,2014,1,nico_rosberg,0,0.0,1,0,0.0,1
1,2014,2,nico_rosberg,1,25.0,1,1,25.0,2
2,2014,3,nico_rosberg,1,43.0,1,2,68.0,1
3,2014,4,nico_rosberg,1,61.0,1,3,111.0,1
4,2014,5,nico_rosberg,1,79.0,1,4,154.0,1


In [7]:
df1 = pd.merge(df, points_before, how='inner', on=['season', 'round', 'driver'])

In [8]:
points_from = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_from': [],
    'constructor_points_from': []
}

In [9]:
grouped = df1.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            n = q.iloc[index]
            points_from['season'].append(season)
            points_from['round'].append(q.iloc[index]['round'])
            points_from['driver'].append(driver)
            points_from['driver_points_from'].append(n.driver_points_after - n.driver_points_before)
            points_from['constructor_points_from'].append(n.constructor_points_after - n.constructor_points_before)
            

points_from = pd.DataFrame(points_from)

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11460/2836361172.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


In [10]:
df2 = pd.merge(df1, points_from, how='inner', on=['season', 'round', 'driver'])

df2[['season', 'round','driver', 'driver_points_before','driver_points_after', 'driver_points_from']].query('season == 2022 & driver == "sebastian_vettel"')

,season,round,driver,driver_points_before,driver_points_after,driver_points_from
3309,2022,3,sebastian_vettel,0.0,0,0.0
3318,2022,4,sebastian_vettel,0.0,4,4.0
3347,2022,5,sebastian_vettel,4.0,4,0.0
3361,2022,6,sebastian_vettel,4.0,4,0.0
3380,2022,7,sebastian_vettel,4.0,5,1.0
3396,2022,8,sebastian_vettel,5.0,13,8.0
3422,2022,9,sebastian_vettel,13.0,13,0.0
3439,2022,10,sebastian_vettel,13.0,15,2.0
3467,2022,11,sebastian_vettel,15.0,15,0.0
3481,2022,12,sebastian_vettel,15.0,15,0.0


In [11]:
more_points = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_per': [],
    'constructor_points_per': [],
    'points_percentage': []
}

In [12]:
grouped = df2.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            n = q.iloc[index]
            more_points['season'].append(season)
            more_points['round'].append(q.iloc[index]['round'])
            more_points['driver'].append(driver)
            more_points['driver_points_per'].append(round(n.driver_points_before / (index + 1), 2))
            more_points['constructor_points_per'].append(round(n.constructor_points_before / (index + 1), 2))
            if n.driver_points_before != 0:
                more_points['points_percentage'].append(round((n.driver_points_before / n.constructor_points_before) * 100, 2))
            else:
                more_points['points_percentage'].append(0.00)
            

more_points = pd.DataFrame(more_points)
more_points.head()

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11460/1308303808.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


,season,round,driver,driver_points_per,constructor_points_per,points_percentage
0,2014,1,nico_rosberg,0.00,0.00,0.00
1,2014,2,nico_rosberg,12.50,12.50,100.00
2,2014,3,nico_rosberg,14.33,22.67,63.24
3,2014,4,nico_rosberg,15.25,27.75,54.95
4,2014,5,nico_rosberg,15.80,30.80,51.30


In [13]:
df3 = pd.merge(df2, more_points, how='inner', on=['season', 'round', 'driver'])
df3.query('season == 2021 & constructor == "mercedes"')

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,driver_points_before,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,constructor_points_from,driver_points_per,constructor_points_per,points_percentage
2812,2021,1,bahrain,lewis_hamilton,mercedes,1,25,1,1,41,...,0.0,1,0,0.0,1,25.0,41.0,0.00,0.00,0.00
2814,2021,1,bahrain,valtteri_bottas,mercedes,3,16,0,3,41,...,0.0,1,0,0.0,1,16.0,41.0,0.00,0.00,0.00
2833,2021,2,imola,lewis_hamilton,mercedes,2,44,1,1,60,...,25.0,1,1,41.0,1,19.0,19.0,12.50,20.50,60.98
2849,2021,2,imola,valtteri_bottas,mercedes,18,16,0,5,60,...,16.0,3,1,41.0,1,0.0,19.0,8.00,20.50,39.02
2852,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,44.0,1,1,60.0,1,25.0,41.0,14.67,20.00,73.33
2854,2021,3,portimao,valtteri_bottas,mercedes,3,32,0,4,101,...,16.0,5,1,60.0,1,16.0,41.0,5.33,20.00,26.67
2872,2021,4,catalunya,lewis_hamilton,mercedes,1,94,3,1,141,...,69.0,1,2,101.0,1,25.0,40.0,17.25,25.25,68.32
2874,2021,4,catalunya,valtteri_bottas,mercedes,3,47,0,3,141,...,32.0,4,2,101.0,1,15.0,40.0,8.00,25.25,31.68
2898,2021,5,monaco,lewis_hamilton,mercedes,7,101,3,2,148,...,94.0,1,3,141.0,1,7.0,7.0,18.80,28.20,66.67
2909,2021,5,monaco,valtteri_bottas,mercedes,19,47,0,4,148,...,47.0,3,3,141.0,1,0.0,7.0,9.40,28.20,33.33


In [14]:
points_last_3 = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_last_3': [],
    'constructor_last_3': []
}

In [15]:
grouped = df3.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            points_last_3['season'].append(season)
            points_last_3['round'].append(q.iloc[index]['round'])
            points_last_3['driver'].append(driver)

            last_3 = list(filter(lambda x : x >= 0, list(range(index-1, index-4, -1))))
            if last_3 == []:
                points_last_3['driver_last_3'].append(0.0)
                points_last_3['constructor_last_3'].append(0.0)
            else:
                n = q.iloc[last_3]
                points_last_3['driver_last_3'].append(n.driver_points_from.sum())
                points_last_3['constructor_last_3'].append(n.constructor_points_from.sum())
                

points_last_3 = pd.DataFrame(points_last_3)

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11460/3230430731.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


In [16]:
df4 = pd.merge(df3, points_last_3, how='inner', on=['season', 'round', 'driver'])
df4.query('season == 2021 & driver == "lewis_hamilton"')

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,constructor_points_from,driver_points_per,constructor_points_per,points_percentage,driver_last_3,constructor_last_3
2812,2021,1,bahrain,lewis_hamilton,mercedes,1,25,1,1,41,...,0,0.0,1,25.0,41.0,0.00,0.00,0.00,0.0,0.0
2833,2021,2,imola,lewis_hamilton,mercedes,2,44,1,1,60,...,1,41.0,1,19.0,19.0,12.50,20.50,60.98,25.0,41.0
2852,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,1,60.0,1,25.0,41.0,14.67,20.00,73.33,44.0,60.0
2872,2021,4,catalunya,lewis_hamilton,mercedes,1,94,3,1,141,...,2,101.0,1,25.0,40.0,17.25,25.25,68.32,69.0,101.0
2898,2021,5,monaco,lewis_hamilton,mercedes,7,101,3,2,148,...,3,141.0,1,7.0,7.0,18.80,28.20,66.67,69.0,100.0
2925,2021,6,baku,lewis_hamilton,mercedes,15,101,3,2,148,...,3,148.0,2,0.0,0.0,16.83,24.67,68.24,57.0,88.0
2932,2021,7,ricard,lewis_hamilton,mercedes,2,119,3,2,178,...,3,148.0,2,18.0,30.0,14.43,21.14,68.24,32.0,47.0
2952,2021,8,red_bull_ring,lewis_hamilton,mercedes,2,138,3,2,212,...,3,178.0,2,19.0,34.0,14.88,22.25,66.85,25.0,37.0
2974,2021,9,red_bull_ring,lewis_hamilton,mercedes,4,150,3,2,242,...,3,212.0,2,12.0,30.0,15.33,23.56,65.09,37.0,64.0
2991,2021,10,silverstone,lewis_hamilton,mercedes,1,177,4,2,285,...,3,242.0,2,27.0,43.0,15.00,24.20,61.98,49.0,94.0


In [17]:
## Removing sprint races

df5 = df4[~
(
    ((df4.season == 2014) & (df4['round'] == 19))
    |
    ((df4.season == 2021) & (df4['round'].isin([10, 14, 19])))
    |
    ((df4.season == 2022) & (df4['round'].isin([4, 11, 21])))
)
]

In [18]:
df6 = df5.drop([
    'constructor_wins_after', 
    'constructor_points_after', 
    'constructor_points_from',
    'driver_points_after', 
    'driver_wins_after', 
    'driver_standings_pos_after',
    'constructor_standings_pos_after'
    ], axis=1)

In [19]:
df6.head()

,season,round,circuit_id,driver,constructor,podium,qualifying_pos,stage,q_delta,driver_wins_before,...,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,driver_points_per,constructor_points_per,points_percentage,driver_last_3,constructor_last_3
0,2014,1,albert_park,nico_rosberg,mercedes,1,3,q3,0.000,0,...,1,0,0.0,1,25.0,0.0,0.0,0.0,0.0,0.0
1,2014,1,albert_park,kevin_magnussen,mclaren,2,4,q3,0.000,0,...,1,0,0.0,1,18.0,0.0,0.0,0.0,0.0,0.0
2,2014,1,albert_park,jenson_button,mclaren,3,11,q2,0.206,0,...,1,0,0.0,1,15.0,0.0,0.0,0.0,0.0,0.0
3,2014,1,albert_park,fernando_alonso,ferrari,4,5,q3,0.000,0,...,1,0,0.0,1,12.0,0.0,0.0,0.0,0.0,0.0
4,2014,1,albert_park,valtteri_bottas,williams,5,10,q3,3.916,0,...,1,0,0.0,1,10.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df6.to_csv(path+'data/processed.csv', index=False)